In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import requests
import pymongo
from datetime import datetime
import ast

In [2]:
file = "data/Atlanta/Atlanta_CRIME_Data.csv"
data1 = pd.read_csv(file)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data1

,Report Number,Report Date,Occur Date,Occur Time,Possible Date,Possible Time,Beat,Apartment Office Prefix,Apartment Number,Location,Shift Occurence,Location Type,UCR Literal,UCR #,IBR Code,Neighborhood,NPU,Latitude,Longitude
0,90010930,1/1/2009,1/1/2009,1145,1/1/2009,1148.0,408.0,NaN,NaN,2841 GREENBRIAR PKWY,Day Watch,8,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68845,-84.49328
1,90011083,1/1/2009,1/1/2009,1330,1/1/2009,1330.0,506.0,NaN,NaN,12 BROAD ST SW,Day Watch,9,LARCENY-NON VEHICLE,630,2303,Downtown,M,33.75320,-84.39201
2,90011208,1/1/2009,1/1/2009,1500,1/1/2009,1520.0,413.0,NaN,NaN,3500 MARTIN L KING JR DR SW,Unknown,8,LARCENY-NON VEHICLE,630,2303,Adamsville,H,33.75735,-84.50282
3,90011218,1/1/2009,1/1/2009,1450,1/1/2009,1510.0,204.0,NaN,NaN,3393 PEACHTREE RD NE,Evening Watch,8,LARCENY-NON VEHICLE,630,2303,Lenox,B,33.84676,-84.36212
4,90011289,1/1/2009,1/1/2009,1600,1/1/2009,1700.0,408.0,NaN,NaN,2841 GREENBRIAR PKWY SW,Unknown,8,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68677,-84.49773
5,90011327,1/1/2009,1/1/2009,1645,1/1/2009,1645.0,607.0,NaN,NaN,1217 CAROLINE ST NE,Evening Watch,24,LARCENY-NON VEHICLE,630,2303,Edgewood,O,33.75786,-84.34875
6,90011450,1/1/2009,1/1/2009,1740,1/1/2009,1815.0,308.0,NaN,NaN,2685 METROPOLITAN PARKWAY,Evening Watch,12,LARCENY-NON VEHICLE,630,2303,Venetian Hills,S,33.70827,-84.45385
7,90011608,1/1/2009,1/1/2009,1627,1/1/2009,2110.0,204.0,NaN,NaN,3393 PEACHTREE RD,Evening Watch,8,LARCENY-NON VEHICLE,630,2303,Lenox,B,33.84676,-84.36212
8,90010721,1/1/2009,1/1/2009,815,1/1/2009,820.0,408.0,NaN,NaN,3030 HEADLAND DR SW,Day Watch,12,LARCENY-NON VEHICLE,630,2303,Greenbriar,R,33.68841,-84.48742
9,90010108,1/1/2009,12/31/2008,2000,1/1/2009,100.0,506.0,NaN,NaN,360 MARIETTA ST NW,Morning Watch,18,LARCENY-FROM VEHICLE,640,2305,Downtown,M,33.76269,-84.39702


In [4]:
# Report Number	Report Date	Occur Date	Occur Time	Possible Date	Possible Time	Beat	
# Apartment Office Prefix	Apartment Number	Location	Shift Occurence	Location Type	
# UCR Literal	UCR #	IBR Code	Neighborhood	NPU	Latitude	Longitude

data = data1[["Report Number", "Occur Date", "Occur Time", "UCR Literal", "Latitude", "Longitude"]].copy()
data.rename(columns={'Latitude':'latitude', 'Longitude':'longitude'}, inplace=True)
data

,Report Number,Occur Date,Occur Time,UCR Literal,latitude,longitude
0,90010930,1/1/2009,1145,LARCENY-NON VEHICLE,33.68845,-84.49328
1,90011083,1/1/2009,1330,LARCENY-NON VEHICLE,33.75320,-84.39201
2,90011208,1/1/2009,1500,LARCENY-NON VEHICLE,33.75735,-84.50282
3,90011218,1/1/2009,1450,LARCENY-NON VEHICLE,33.84676,-84.36212
4,90011289,1/1/2009,1600,LARCENY-NON VEHICLE,33.68677,-84.49773
5,90011327,1/1/2009,1645,LARCENY-NON VEHICLE,33.75786,-84.34875
6,90011450,1/1/2009,1740,LARCENY-NON VEHICLE,33.70827,-84.45385
7,90011608,1/1/2009,1627,LARCENY-NON VEHICLE,33.84676,-84.36212
8,90010721,1/1/2009,815,LARCENY-NON VEHICLE,33.68841,-84.48742
9,90010108,12/31/2008,2000,LARCENY-FROM VEHICLE,33.76269,-84.39702


In [6]:
dates = []
years = []

for date in data["Occur Date"]:
    date = datetime.strptime(date, "%m/%d/%Y")
    dates.append(date)
    years.append(date.year)
    
data["date"] = dates
data["years"] = years
data

,Report Number,Occur Date,Occur Time,UCR Literal,latitude,longitude,date,years
0,90010930,1/1/2009,1145,LARCENY-NON VEHICLE,33.68845,-84.49328,2009-01-01,2009
1,90011083,1/1/2009,1330,LARCENY-NON VEHICLE,33.75320,-84.39201,2009-01-01,2009
2,90011208,1/1/2009,1500,LARCENY-NON VEHICLE,33.75735,-84.50282,2009-01-01,2009
3,90011218,1/1/2009,1450,LARCENY-NON VEHICLE,33.84676,-84.36212,2009-01-01,2009
4,90011289,1/1/2009,1600,LARCENY-NON VEHICLE,33.68677,-84.49773,2009-01-01,2009
5,90011327,1/1/2009,1645,LARCENY-NON VEHICLE,33.75786,-84.34875,2009-01-01,2009
6,90011450,1/1/2009,1740,LARCENY-NON VEHICLE,33.70827,-84.45385,2009-01-01,2009
7,90011608,1/1/2009,1627,LARCENY-NON VEHICLE,33.84676,-84.36212,2009-01-01,2009
8,90010721,1/1/2009,815,LARCENY-NON VEHICLE,33.68841,-84.48742,2009-01-01,2009
9,90010108,12/31/2008,2000,LARCENY-FROM VEHICLE,33.76269,-84.39702,2008-12-31,2008


In [8]:
data_2014 = pd.DataFrame()
data_2015 = pd.DataFrame()
data_2016 = pd.DataFrame()
data_2017 = pd.DataFrame()
data_2018 = pd.DataFrame()

data_2014 = data.loc[data["years"] == 2014]
print(data_2014)
data_2015 = data.loc[data["years"] == 2015]
print(data_2015)
data_2016 = data.loc[data["years"] == 2016]
print(data_2016)
data_2017 = data.loc[data["years"] == 2017]
print(data_2017)
data_2018 = data.loc[data["years"] == 2018]
data_2018

        Report Number  Occur Date  Occur Time          UCR Literal  latitude  \
175621      140011254    1/1/2014         100           AUTO THEFT  33.69915   
175622      140012074    1/1/2014        1940   ROBBERY-PEDESTRIAN  33.75290   
175623      140010429    1/1/2014         226   ROBBERY-PEDESTRIAN  33.77575   
175624      140010494    1/1/2014         245          AGG ASSAULT  33.76244   
175625      140011847    1/1/2014        1750          AGG ASSAULT  33.78554   
...               ...         ...         ...                  ...       ...   
248423      161560078    6/4/2014          27  LARCENY-NON VEHICLE  33.75861   
254034      162242355   8/11/2014        1800   BURGLARY-RESIDENCE  33.75127   
289911      173352433  11/24/2014        2000  LARCENY-NON VEHICLE  33.84676   
298767      181011366   11/5/2014        1200  LARCENY-NON VEHICLE  33.81924   
304065      181750378   6/23/2014        2345  LARCENY-NON VEHICLE  33.84100   

        longitude       date  years  
1

,Report Number,Occur Date,Occur Time,UCR Literal,latitude,longitude,date,years
292276,180011286,1/1/2018,330,LARCENY-FROM VEHICLE,33.69811,-84.42947,2018-01-01,2018
292281,180011390,1/1/2018,1530,LARCENY-NON VEHICLE,33.73604,-84.43627,2018-01-01,2018
292282,180011692,1/1/2018,1625,LARCENY-NON VEHICLE,33.85182,-84.35950,2018-01-01,2018
292283,180011735,1/1/2018,1700,LARCENY-NON VEHICLE,33.65722,-84.51029,2018-01-01,2018
292287,180011167,1/1/2018,30,AUTO THEFT,33.78887,-84.42027,2018-01-01,2018
292288,180011188,1/1/2018,130,AUTO THEFT,33.78574,-84.44400,2018-01-01,2018
292292,180011737,1/1/2018,1400,AUTO THEFT,33.75619,-84.44449,2018-01-01,2018
292293,180011778,1/1/2018,1000,AUTO THEFT,33.75767,-84.39266,2018-01-01,2018
292294,180011858,1/1/2018,1330,AUTO THEFT,33.75860,-84.36513,2018-01-01,2018
292295,180011490,1/1/2018,1634,ROBBERY-PEDESTRIAN,33.70433,-84.38079,2018-01-01,2018


In [11]:
def reformat_datetime(df):
    datetimes = []
    for d,t  in zip(df["date"], df["Occur Time"]):
#         print(d)
#         print(type(d))
    #     datetime = (datetime.strftime(d, format='%Y-%m-%d') + " " + str(t))
        d = datetime.strptime(str(d), "%Y-%m-%d %H:%M:%S")
        date_time = datetime.strftime(d, format='%Y-%m-%d') + "T" + str((int(t/100))) + ":00:00"
    #     datetime = d + " " + t
        datetimes.append(date_time)
    df["date_time"] = datetimes
    return df

# def reformat_datetime(df):
#     datetimes = []
#     for d in df["date"]:
#         date_time = datetime.strftime(d, format='%Y-%m-%dT%H:00:00')
#         datetimes.append(date_time)
#     df["date_time"] = datetimes
#     return df

In [12]:
df_years = [data_2014, data_2015, data_2016, data_2017, data_2018]

for i, yr in enumerate(df_years):
    df = reformat_datetime(yr)
    if i == 0:
        data_2014_1 = df.copy()
    elif i ==1:
        data_2015_1 = df.copy()
    elif i ==2:
        data_2016_1 = df.copy()
    elif i ==3:
        data_2017_1 = df.copy()
    elif i ==4:
        data_2018_1 = df.copy()


/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()





----

# Final view of date_time into weather data datetime format

In [13]:
data_2014_1

,Report Number,Occur Date,Occur Time,UCR Literal,latitude,longitude,date,years,date_time
175621,140011254,1/1/2014,100,AUTO THEFT,33.69915,-84.41819,2014-01-01,2014,2014-01-01T1:00:00
175622,140012074,1/1/2014,1940,ROBBERY-PEDESTRIAN,33.75290,-84.43904,2014-01-01,2014,2014-01-01T19:00:00
175623,140010429,1/1/2014,226,ROBBERY-PEDESTRIAN,33.77575,-84.46804,2014-01-01,2014,2014-01-01T2:00:00
175624,140010494,1/1/2014,245,AGG ASSAULT,33.76244,-84.41272,2014-01-01,2014,2014-01-01T2:00:00
175625,140011847,1/1/2014,1750,AGG ASSAULT,33.78554,-84.45441,2014-01-01,2014,2014-01-01T17:00:00
175626,140012191,1/1/2014,2000,AGG ASSAULT,33.75508,-84.45825,2014-01-01,2014,2014-01-01T20:00:00
175627,140010421,1/1/2014,215,AGG ASSAULT,33.70296,-84.45512,2014-01-01,2014,2014-01-01T2:00:00
175628,140010647,1/1/2014,410,AGG ASSAULT,33.75922,-84.44260,2014-01-01,2014,2014-01-01T4:00:00
175629,140010377,1/1/2014,200,AGG ASSAULT,33.84779,-84.36633,2014-01-01,2014,2014-01-01T2:00:00
175633,140010692,1/1/2014,100,BURGLARY-RESIDENCE,33.79333,-84.47416,2014-01-01,2014,2014-01-01T1:00:00


In [14]:
data_2015_1

,Report Number,Occur Date,Occur Time,UCR Literal,latitude,longitude,date,years,date_time
206755,150010872,1/1/2015,540,BURGLARY-RESIDENCE,33.74577,-84.31650,2015-01-01,2015,2015-01-01T5:00:00
206756,150012144,1/1/2015,2002,BURGLARY-RESIDENCE,33.68066,-84.49371,2015-01-01,2015,2015-01-01T20:00:00
206757,150012260,1/1/2015,1630,BURGLARY-RESIDENCE,33.66657,-84.39138,2015-01-01,2015,2015-01-01T16:00:00
206758,150010468,1/1/2015,215,AGG ASSAULT,33.76139,-84.38406,2015-01-01,2015,2015-01-01T2:00:00
206759,150011023,1/1/2015,818,AGG ASSAULT,33.69867,-84.49750,2015-01-01,2015,2015-01-01T8:00:00
206761,150010419,1/1/2015,150,AGG ASSAULT,33.71759,-84.42831,2015-01-01,2015,2015-01-01T1:00:00
206762,150010601,1/1/2015,300,AGG ASSAULT,33.76929,-84.38554,2015-01-01,2015,2015-01-01T3:00:00
206763,150011828,1/1/2015,1200,LARCENY-FROM VEHICLE,33.74667,-84.40285,2015-01-01,2015,2015-01-01T12:00:00
206765,150012140,1/1/2015,100,LARCENY-FROM VEHICLE,33.76319,-84.41612,2015-01-01,2015,2015-01-01T1:00:00
206766,150012147,1/1/2015,2000,LARCENY-FROM VEHICLE,33.84676,-84.36212,2015-01-01,2015,2015-01-01T20:00:00


In [15]:
data_2016_1

,Report Number,Occur Date,Occur Time,UCR Literal,latitude,longitude,date,years,date_time
236844,160011550,1/1/2016,1420,LARCENY-NON VEHICLE,33.84676,-84.36212,2016-01-01,2016,2016-01-01T14:00:00
236845,160011569,1/1/2016,1449,LARCENY-NON VEHICLE,33.75674,-84.34619,2016-01-01,2016,2016-01-01T14:00:00
236846,160011645,1/1/2016,1530,LARCENY-NON VEHICLE,33.75144,-84.44382,2016-01-01,2016,2016-01-01T15:00:00
236847,160011823,1/1/2016,1515,LARCENY-NON VEHICLE,33.84676,-84.36212,2016-01-01,2016,2016-01-01T15:00:00
236848,160011975,1/1/2016,15,LARCENY-NON VEHICLE,33.75082,-84.39623,2016-01-01,2016,2016-01-01T0:00:00
236849,160011958,1/1/2016,1805,LARCENY-NON VEHICLE,33.70409,-84.35789,2016-01-01,2016,2016-01-01T18:00:00
236850,160012284,1/1/2016,2016,ROBBERY-PEDESTRIAN,33.71687,-84.41063,2016-01-01,2016,2016-01-01T20:00:00
236851,160011938,1/1/2016,1750,ROBBERY-COMMERCIAL,33.75176,-84.49182,2016-01-01,2016,2016-01-01T17:00:00
236853,160010061,1/1/2016,3,AGG ASSAULT,33.72209,-84.43107,2016-01-01,2016,2016-01-01T0:00:00
236854,160010849,1/1/2016,503,AGG ASSAULT,33.76846,-84.37198,2016-01-01,2016,2016-01-01T5:00:00


In [16]:
data_2017_1

,Report Number,Occur Date,Occur Time,UCR Literal,latitude,longitude,date,years,date_time
265865,170010079,1/1/2017,0,LARCENY-FROM VEHICLE,33.68990,-84.50584,2017-01-01,2017,2017-01-01T0:00:00
265870,170010712,1/1/2017,130,LARCENY-FROM VEHICLE,33.81038,-84.36122,2017-01-01,2017,2017-01-01T1:00:00
265873,170011070,1/1/2017,1025,LARCENY-FROM VEHICLE,33.74505,-84.40523,2017-01-01,2017,2017-01-01T10:00:00
265876,170011255,1/1/2017,1200,LARCENY-FROM VEHICLE,33.72602,-84.32324,2017-01-01,2017,2017-01-01T12:00:00
265879,170011504,1/1/2017,1500,LARCENY-FROM VEHICLE,33.76341,-84.35289,2017-01-01,2017,2017-01-01T15:00:00
265880,170011525,1/1/2017,1537,LARCENY-FROM VEHICLE,33.73760,-84.39486,2017-01-01,2017,2017-01-01T15:00:00
265881,170011531,1/1/2017,1415,LARCENY-FROM VEHICLE,33.76408,-84.31645,2017-01-01,2017,2017-01-01T14:00:00
265883,170011751,1/1/2017,1530,LARCENY-FROM VEHICLE,33.75832,-84.38905,2017-01-01,2017,2017-01-01T15:00:00
265885,170012028,1/1/2017,1430,LARCENY-FROM VEHICLE,33.75916,-84.40771,2017-01-01,2017,2017-01-01T14:00:00
265886,170012145,1/1/2017,2030,LARCENY-FROM VEHICLE,33.85547,-84.38135,2017-01-01,2017,2017-01-01T20:00:00


In [17]:
data_2018_1

,Report Number,Occur Date,Occur Time,UCR Literal,latitude,longitude,date,years,date_time
292276,180011286,1/1/2018,330,LARCENY-FROM VEHICLE,33.69811,-84.42947,2018-01-01,2018,2018-01-01T3:00:00
292281,180011390,1/1/2018,1530,LARCENY-NON VEHICLE,33.73604,-84.43627,2018-01-01,2018,2018-01-01T15:00:00
292282,180011692,1/1/2018,1625,LARCENY-NON VEHICLE,33.85182,-84.35950,2018-01-01,2018,2018-01-01T16:00:00
292283,180011735,1/1/2018,1700,LARCENY-NON VEHICLE,33.65722,-84.51029,2018-01-01,2018,2018-01-01T17:00:00
292287,180011167,1/1/2018,30,AUTO THEFT,33.78887,-84.42027,2018-01-01,2018,2018-01-01T0:00:00
292288,180011188,1/1/2018,130,AUTO THEFT,33.78574,-84.44400,2018-01-01,2018,2018-01-01T1:00:00
292292,180011737,1/1/2018,1400,AUTO THEFT,33.75619,-84.44449,2018-01-01,2018,2018-01-01T14:00:00
292293,180011778,1/1/2018,1000,AUTO THEFT,33.75767,-84.39266,2018-01-01,2018,2018-01-01T10:00:00
292294,180011858,1/1/2018,1330,AUTO THEFT,33.75860,-84.36513,2018-01-01,2018,2018-01-01T13:00:00
292295,180011490,1/1/2018,1634,ROBBERY-PEDESTRIAN,33.70433,-84.38079,2018-01-01,2018,2018-01-01T16:00:00


In [18]:
# .to_csv(index=False)
data_2014_1.to_csv (r'data/Atlanta/data_2014.csv', index = None, header=True)
data_2015_1.to_csv (r'data/Atlanta/data_2015.csv', index = None, header=True)
data_2016_1.to_csv (r'data/Atlanta/data_2016.csv', index = None, header=True)
data_2017_1.to_csv (r'data/Atlanta/data_2017.csv', index = None, header=True)
data_2018_1.to_csv (r'data/Atlanta/data_2018.csv', index = None, header=True)